In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

# Problem 6 - Identify patterns in stock data

This notebook's goal is to present a solution for the proposed problem 6. The applied methodology was a simplified CRISP-DS.

## 1. Business Understanding

Identifying patterns in stock data can lead to better investment decisions, which result in a better profit. The goal of this challenge is to, given a Toronto-Dominion Bank stock candlestick time series chart, point other time series that show similar behavior.

The selected window contains data from 01/12/2021 to 01/02/2022, and it is shown below.

![reference](images/to_td.png)

## 2. Data Understanding

The desired pattern is shown on a candlestick chart. This chart summarizes information about the highest price, lowest price, opening price and closing price of a stock in a chosen resolution, in this challenge, daily.

An example is shown on the image below, obtained from reference [1].

![candlestick](images/candlestick.png)

A sample of the available data to search similar patterns is shown below. We can see that the rows contain daily data about several stocks from different exchanges.

In [2]:
data = pq.read_table(
  'data/pyarrow_database.parquet',
  columns = ['Date', 'Open', 'Close', 'Ticker', 'Exchange',
             'High', 'Low', 'Adj Close', 'Volume']) \
  .to_pandas()
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data.set_index(['Exchange', 'Ticker', 'Date'], inplace=True)
data.head()

Open      Close       High        Low  \
Exchange Ticker Date                                                     
NYSE     A      2008-09-02  25.100143  24.670959  25.536482  24.527897   
                2008-09-03  24.570814  24.184549  24.678112  23.862661   
                2008-09-04  23.977110  23.018599  23.984262  22.989986   
                2008-09-05  22.889843  23.454935  23.547926  22.711016   
                2008-09-08  23.876966  23.583691  23.977110  23.297567   

                            Adj Close     Volume  
Exchange Ticker Date                              
NYSE     A      2008-09-02  22.517595  2876665.0  
                2008-09-03  22.073639  3855265.0  
                2008-09-04  21.009457  3656609.0  
                2008-09-05  21.407715  3217357.0  
                2008-09-08  21.525227  3046102.0

## 3. Data Preparation

As a first iteration, we decide to work with a new variable named 'variation', that lossly compresses information seen in the candlestick chart, but allow us to perform a univariate analysis. A sample is shown below.

In [3]:
data = data[['Open', 'Close']]
data['variation'] = (data['Close'] - data['Open'])/data['Open']

data.drop(columns=['Open', 'Close'], inplace=True)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)
data.head()

variation
Exchange Ticker Date                 
NYSE     A      2008-09-02  -0.017099
                2008-09-03  -0.015720
                2008-09-04  -0.039976
                2008-09-05   0.024687
                2008-09-08  -0.012283

## 4. Modeling

Before aiming for complex machine learning algorithms, we want to be sure that classic algorithms won't provide a good enough result. So, as a first iteration, we compare a rolling window of the 'variation' feature to every possible window on the dataset. For each pair of windows, we obtain the errors by an element-wise subtraction, and then calculate the RMSE.

We note that this approach is computationally expensive, as the solution has big O(n²) complexity. But the machine learning alternatives would also require a lot of computational resources for training. In summary, given the large dataset, both classic and machine learning solutions require a lot of processing time.

In [4]:
def search_pattern(series, pattern):
    start_date = []
    end_date = []
    results = []
    reps = len(series)- len(pattern) + 1
    for i in range(0, reps):
        current_window = series.iloc[i:i+len(pattern)]
        start_date.append(series.index[i])
        end_date.append(series.index[i+len(pattern)-1])
        results.append(mean_squared_error(current_window, pattern, squared=False))
    return pd.Series(results, [start_date, end_date])

In [5]:
## Avoid running a expensive evaluation multiple times
if False:
  pattern = data.loc[('TO', 'TD')].loc['2021-12-01':'2022-02-01']
  
  results = data.groupby(['Exchange', 'Ticker']).apply(search_pattern, pattern)
  results.sort_values(ascending=True) \
    .head(1000) \
    .groupby(level=['Exchange', 'Ticker'], sort=False) \
    .first() \
    .to_csv('data/most_similiar_windows.csv')
  del results

## 5. Evaluation

To make easier to compare the similarity between the windows, a score that ranges from 0 to 100 is calculated.

In [6]:
most_similiar_windows = pd.read_csv(
  'data/most_similiar_windows.csv',
  index_col=['Exchange', 'Ticker', 'Start', 'End'])

min_error = most_similiar_windows['score'][1]
max_error = most_similiar_windows['score'][-1]

most_similiar_windows['100_based_score'] = (most_similiar_windows['score'] * 100)/min_error
excess_error = most_similiar_windows['100_based_score'] - 100
most_similiar_windows['100_based_score'] = most_similiar_windows['100_based_score'] - 2 * excess_error
most_similiar_windows = most_similiar_windows.iloc[1: , :]

most_similiar_windows.head()

score  \
Exchange Ticker Start                                            End                                                          
TO       XFN    ('TO', 'XFN', Timestamp('2021-12-01 00:00:00'))  ('TO', 'XFN', Timestamp('2022-02-01 00:00:00'))   0.005363   
         RBNK   ('TO', 'RBNK', Timestamp('2021-12-01 00:00:00')) ('TO', 'RBNK', Timestamp('2022-02-01 00:00:00'))  0.006059   
         HCA    ('TO', 'HCA', Timestamp('2021-12-01 00:00:00'))  ('TO', 'HCA', Timestamp('2022-02-01 00:00:00'))   0.006121   
         CIC    ('TO', 'CIC', Timestamp('2021-12-01 00:00:00'))  ('TO', 'CIC', Timestamp('2022-02-01 00:00:00'))   0.006508   
         XDIV   ('TO', 'XDIV', Timestamp('2021-12-01 00:00:00')) ('TO', 'XDIV', Timestamp('2022-02-01 00:00:00'))  0.006552   

                                                                                                                   100_based_score  
Exchange Ticker Start                                            End                                                                
TO       XFN    ('TO', 'XFN', Timestamp('2021-12-01 00:00:00'))  ('TO', 'XFN', Timestamp('2022-02-01 00:00:00'))        100.000000  
         RBNK   ('TO', 'RBNK', Timestamp('2021-12-01 00:00:00')) ('TO', 'RBNK', Timestamp('2022-02-01 00:00:00'))        87.021244  
         HCA    ('TO', 'HCA', Timestamp('2021-12-01 00:00:00'))  ('TO', 'HCA', Timestamp('2022-02-01 00:00:00'))         85.855628  
         CIC    ('TO', 'CIC', Timestamp('2021-12-01 00:00:00'))  ('TO', 'CIC', Timestamp('2022-02-01 00:00:00'))         78.644520  
         XDIV   ('TO', 'XDIV', Timestamp('2021-12-01 00:00:00')) ('TO', 'XDIV', Timestamp('2022-02-01 00:00:00'))        77.824516

The images for the benchmark, Toronto-Dominion Bank in the specified window, are compared to the top 5 most similar stocks. We can visually notice the similarity, but we also highlight that:

    1. All the five most similar windows match the window for the benchmark;
    2. All the five most similar stocks are from negotiated in Toronto Stock Exchange;
    3. All the five most similar stocks are, in fact, Exchange-Traded Fund (ETF).
    
From the information above, we understand that the high correlation between the benchmark window and the top 5 similar windows occur because the Toronto-Dominion Bank is contained in those ETFs strategies, so the variation between them should be correlated.

The charts from the top 5 scores are shown below.

![reference](images/etfs/to_xfn.png)
![reference](images/etfs/to_rbnk.png)
![reference](images/etfs/to_hca.png)
![reference](images/etfs/to_cic.png)
![reference](images/etfs/to_xdiv.png)

To check the algorithms' performance on stocks that aren't correlated, we selected a selected stocks from other Exchange markets, such as NYSE and NASDAQ. We note that NYSE BRO show a similar pattern, but NASDAQ GYRO and EBMT results are not very clear.

![reference](images/misc/nyse_bro.png)
![reference](images/misc/nasdaq_gyro.png)
![reference](images/misc/nasdaq_ebmt.png)

## 6. Conclusion

The classic programming methods show to be enough for this solution. Obviously there are some caveats, such as: 

    1. Computational cost, that will occur every time we need to search for a new pattern;
    2. The lack of flexibility in the window size. Patterns that are similar but slightly shorter or longer won't be recognized.
    
But, in general, we note that the charts obtained as result are visually similar to the desired pattern, concluding that the solution has succeeded.

## References

1. [Introduction to Candlestick Chart](https://www.investopedia.com/trading/candlestick-charting-what-is-it/)